何をもって俊敏さを評価するか  

平均速度、最大速度[deg/ms]  
平均加速度、最大加速度  
平均躍度、最大躍度
サッケード回数[回]  
方向転換の頻度[回/ms]  
停止時間の合計[ms]  
速度V以上の時間の合計[ms]  
移動距離の合計[deg]  
サッケード中の移動距離の合計[deg]  
FFT(高周波成分が強いほど、視線の動きが速く変化していることを示す)

In [69]:
import sys
sys.path.append('G:/共有ドライブ/MCI/v2/script/src')
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.expand_frame_repr', False) # DataFrameの表示を折り返さない

In [70]:
import sys
import os

# パスの設定 - 画像から見えるパス構造に基づく
src_path = 'G:/共有ドライブ/MCI/v2/script/src'

# パスが存在するか確認と追加
if os.path.exists(src_path):
    sys.path.append(src_path)
    print(f"Added path: {src_path}")
else:
    print(f"Warning: Path does not exist: {src_path}")

try:
    # .pycファイルが存在するので、モジュール名だけで importできるはず
    import mciutil
    import velocity_algorithm
    print("Successfully imported modules")
except ImportError as e:
    print(f"Import error: {e}")
    print("Current sys.path:")
    for p in sys.path:
        print(f"  {p}")

Added path: G:/共有ドライブ/MCI/v2/script/src
Successfully imported modules


In [71]:
# CSVファイルの内容を確認
file_path = "G:/共有ドライブ/GAP_長寿研/user/iwamoto/20241009101605_1/calibration_1_20241009_20241119145059.csv"
df = pd.read_csv(file_path)

# カラム名の表示
print("列名一覧:")
print(df.columns.tolist())

# 最初の数行を表示
print("\nデータの最初の5行:")
print(df.head())

列名一覧:
['Index', 'CalibrationCount', 'EyeSide', 'CalibrationPointerX', 'CalibrationPointerY', 'DrawPointerX', 'DrawPointerY', 'timestamp', 'EyeCenterX', 'EyeCenterY', 'RotatedEyeCenterX', 'RotatedEyeCenterY', 'IsFixed', 'RetryCount', 'IsBadPosition', 'IsValid', 'PupilSize', 'PupilAspect', 'Phase', 'EyeCenterAngleX', 'EyeCenterAngleY']

データの最初の5行:
   Index  CalibrationCount  EyeSide  CalibrationPointerX  CalibrationPointerY  DrawPointerX  DrawPointerY  timestamp  EyeCenterX  EyeCenterY  ...  RotatedEyeCenterY  IsFixed  RetryCount  IsBadPosition  IsValid  PupilSize  PupilAspect  Phase  EyeCenterAngleX  EyeCenterAngleY
0      0                 1        1                  NaN                  NaN           0.0           0.0         19         302         223  ...           106.2614        0         NaN              0        0         51          720      1         0.776040         1.559823
1      1                 1        1                  NaN                  NaN           0.0           

In [72]:
# 必要なカラム名の追加とデータの前処理
processed_df = df.copy()

# CloseEyeカラムを追加（IsValidを基に設定：IsValid=0の場合を瞬きとみなす）
processed_df['CloseEye'] = processed_df['IsValid'].apply(lambda x: 1 if x == 0 else 0)

# velocity_based_algorithm_summaryの呼び出し
frame_df, time_df = velocity_based_algorithm_summary(
    processed_df,  # 処理済みデータフレーム
    'EyeCenterAngleX',  # x座標のカラム名
    'EyeCenterAngleY',  # y座標のカラム名
    threshold=0.0003  # 閾値
)

# 結果の確認
print("\n解析結果の最初の5行:")
print(frame_df.head())


解析結果の最初の5行:
   timestamp    gaze_x    gaze_y  velocities  is_fixations  fixation_indices
0         19  0.776040  1.559823    0.000000         False                 0
1         37  0.776040  1.559823    0.135905         False                 0
2         57  0.916767  1.034734    0.000000         False                 0
3         72  0.916767  1.034734    0.135905         False                 0
4         88  0.776040  1.559823    0.135905         False                 0


In [73]:
# 必要なカラム名の追加とデータの前処理
processed_df = df.copy()

# CloseEyeカラムを追加（IsValidを基に設定）
processed_df['CloseEye'] = processed_df['IsValid'].apply(lambda x: 1 if x == 0 else 0)

# velocity_based_algorithm_summaryの呼び出し
frame_df, time_df = velocity_based_algorithm_summary(
    processed_df,  # 処理済みデータフレーム
    'EyeCenterAngleX',  # x座標のカラム名
    'EyeCenterAngleY',  # y座標のカラム名
    threshold=0.0003  # 閾値
)

In [74]:
# データの読み込みとクレンジング
df = get_cleansed_df("G:/共有ドライブ/GAP_長寿研/user/iwamoto/20241009101605_1/calibration_1_20241009_20241119145059.csv")

# 速度ベースの分析実行
frame_df, time_df = velocity_based_algorithm_summary(
    df,
    'EyeCenterAngleX',
    'EyeCenterAngleY',
    threshold=0.0003
)

In [78]:
# データフレームの列名を確認
print("データフレームのカラム名:")
print(df.columns.tolist())

# 最初の数行を表示して構造を確認
print("\nデータの最初の5行:")
print(df.head())

データフレームのカラム名:
['Index', 'CalibrationCount', 'EyeSide', 'CalibrationPointerX', 'CalibrationPointerY', 'DrawPointerX', 'DrawPointerY', 'timestamp', 'EyeCenterX', 'EyeCenterY', 'RotatedEyeCenterX', 'RotatedEyeCenterY', 'IsFixed', 'RetryCount', 'IsBadPosition', 'IsValid', 'PupilSize', 'PupilAspect', 'Phase', 'EyeCenterAngleX', 'EyeCenterAngleY']

データの最初の5行:
   Index  CalibrationCount  EyeSide  CalibrationPointerX  CalibrationPointerY  DrawPointerX  DrawPointerY  timestamp  EyeCenterX  EyeCenterY  ...  RotatedEyeCenterY  IsFixed  RetryCount  IsBadPosition  IsValid  PupilSize  PupilAspect  Phase  EyeCenterAngleX  EyeCenterAngleY
0      0                 1        1                  NaN                  NaN           0.0           0.0         19         302         223  ...           106.2614        0         NaN              0        0         51          720      1         0.776040         1.559823
1      1                 1        1                  NaN                  NaN           0.0   

In [79]:
import sys
sys.path.append('G:/共有ドライブ/MCI/v2/script/src')
from mciutil import get_cleansed_df

In [80]:
import sys
sys.path.append('G:/共有ドライブ/MCI/v2/script/src')
from mciutil import get_cleansed_df
from velocity_algorithm import velocity_based_algorithm_summary

# データの読み込みとクレンジング
df = get_cleansed_df("G:/共有ドライブ/GAP_長寿研/user/iwamoto/20241009101605_1/calibration_1_20241009_20241119145059.csv")

# 速度ベースの分析実行
frame_df, time_df = velocity_based_algorithm_summary(df)

KeyError: 'movie_pos_x'

In [81]:
# vscodeで実行
path = 'G:/共有ドライブ/MCI/v2'

In [82]:
dir = 'kyoto'
period = '202409'
groups = [
    'control',
    'mci',
    ]

import_path = f'{path}/output/v2.1/{dir}/{period}/0th_analysis'

In [83]:
mci_subtype = pd.read_csv(r'G:\共有ドライブ\MCI\v2\master\old\mci_subtype_mmse.csv')

mci_subtype_list = list(mci_subtype['subtype'].unique())
mci_subtype_list

['Others', 'Unknown', 'Mixed', 'AD', nan, 'LBD', 'Vascular', 'iNPH']

In [84]:
mci_subtype.head(n=5)


,FINDEXID,subtype,mmse
0,2_20046,Others,27.0
1,2_20047,Unknown,30.0
2,2_20049,Mixed,20.0
3,2_20117,AD,25.0
4,2_20124,Others,20.0


In [85]:
import glob
import numpy as np
import time

cleansing_parameters = pd.DataFrame()
start_time = time.time()
group_paths = glob.glob(import_path+'/*')
group_paths = [x.replace("\\","/") for x in group_paths]
group_paths = [path for path in group_paths if any(group_name in path for group_name in groups)]
dfs = pd.DataFrame()

for index, group in enumerate(group_paths):
    group_name = group.split('/')[-1]
    print(f'group name: {group_name}')

    files = glob.glob(f'{import_path}/{group_name}/*.csv')
    files = [x.replace("\\","/") for x in files]
    print(len(files))
    for inspection_dir in files:
        ins_name = inspection_dir.split('/')[-1].split('.')[0]
        if int(ins_name.split("_")[0]) == 4:
            id = ins_name.split("_")[0] + "_" + ins_name.split("_")[1] + "_" + ins_name.split("_")[2]
        else:
            id = ins_name.split("_")[0] + "_" + ins_name.split("_")[1]
        inspection_date = int(ins_name.split('_')[-1])
        '''
        現行特徴量と比較するため、サブタイプの有無は考慮しない
        '''
        # if group_name == 'control':
        #     df_index = 0
        #     subtype = np.nan
        #     mmse = np.nan

        # if group_name == 'mci':
        #     matched_line = mci_subtype[mci_subtype['FINDEXID'] == id]
        #     if len(matched_line) == 0:
        #         print('a')
        #         continue
        #     subtype = mci_subtype[mci_subtype['FINDEXID'] == id]['subtype'].iloc[0]
        #     mmse = mci_subtype[mci_subtype['FINDEXID'] == id]['mmse'].iloc[0]
        #     df_index = mci_subtype_list.index(subtype) + 1
        df = get_cleansed_df(inspection_dir)
        df["id"] = ins_name
        df["group_name"] = group_name.upper()
        # df["subtype"] = subtype
        # df["mmse"] = mmse
        dfs = pd.concat([dfs, df])
dfs.shape

group name: control
125
group name: mci
80


(4510205, 5)

In [ ]:
df.head(n=5)

NameError: name 'df' is not defined

In [ ]:
dfs.head()

""


In [ ]:
ids = dfs["id"].unique()
len(ids)

KeyError: 'id'

# -----rawdataの用意完了-----

In [ ]:
def get_velocity_info(df): # 速度の平均、最大値
    velo_mean = df['velocities'].mean()
    velo_max = df['velocities'].max()
    return velo_mean, velo_max

def get_acceleration_info(df): # 加速度の平均、最大値
    acce_mean = df['velocities'].diff().mean()
    acce_max = df['velocities'].diff().max()
    return acce_mean, acce_max

def get_jerk_info(df):  # 躍度の平均、最大値
    dt = 4  # sec/frame
    
    # 時間間隔を考慮した躍度の計算
    acceleration = df['velocities'].diff() / dt
    jerk = acceleration.diff() / dt
    
    
    # NaNを除外して統計量を計算
    jerk_clean = jerk.dropna()
    jerk_mean = jerk.mean()
    jerk_max = jerk.max()
    jerk_min = jerk_clean.min()  # 負の躍度も重要な情報
    
    return jerk_mean, jerk_max

def get_saccade_count(df): # サッケード回数
    count = df['fixation_indices'].max()
    # 最後が固視で終了していたら
    if not pd.isna(df['fixation_indices'].iloc[-1]):
        count -= 1
    return count

def get_turn_count(df):
    count = 0
    return count

def get_fixtaion_time_sum(df): # 止まっている判定されている時間の合計
    dt = 4 # sec/frame
    time_sum = len(df[df['is_fixations'] == True]['timestamp']) * dt
    return time_sum

def get_move_time_sum(df): # 一定以上の速度が出ている時間の合計
    threshold = 0.001
    dt = 4 # sec/frame
    time_sum = len(df[df['velocities'] > threshold]['timestamp']) * dt
    return time_sum

def get_move_dis_sum(df): # 移動距離の合計
    dis_sum = df['velocities'].sum()
    return dis_sum

def get_saccade_move_dis_sum(df): # 止まっていないときの移動距離の合計
    dis_sum = df[df['is_fixations'] == False]['velocities'].sum()
    return dis_sum

def get_velo_fft(df):
    return None

In [ ]:
import matplotlib.pyplot as plt

def show_graph(df_selected, params, p_index):
    fig = plt.figure(figsize=(10,5))
    fig.suptitle(f"{group_name} id:{id} param:{params[p_index][4]}")
    ax1 = fig.add_subplot(1,2,1)
    ax1.set_title("velocity")
    ax1.plot(df_selected["timestamp"], df_selected["velocities"], label='velocities')
    ax1.plot(df_selected["timestamp"], df_selected["velocities"].diff(), label='acceleration')
    ax1.set_ylim(0,0.007)
    ax1.legend()

    ax2 = fig.add_subplot(1,2,2)
    ax2.set_title("軌跡")
    ax2.scatter(df_selected['movie_pos_x'], df_selected['movie_pos_y'], s=0.5, c='gray', alpha=0.3)
    ax2.plot(df_selected['movie_pos_x_interpolated'], df_selected['movie_pos_y_interpolated'], c='green', alpha=0.3)
    ax2.scatter(df_selected[df_selected['is_fixations'] == True]['movie_pos_x'], df_selected[df_selected['is_fixations'] == True]['movie_pos_y'], s=0.5, c='blue', alpha=1)
    ax2.scatter(df_selected['movie_pos_x'].dropna().iloc[0], df_selected['movie_pos_y'].dropna().iloc[0], s=50, c='black', marker='x') # スタート
    ax2.scatter(df_selected['movie_pos_x'].dropna().iloc[-1], df_selected['movie_pos_y'].dropna().iloc[-1], s=50, c='red', marker='x') # ラスト
    ax2.scatter(params[p_index][2], params[p_index][3], color='red') # 視標
    ax2.set_xlim(0,1)
    ax2.set_ylim(1,0)
    plt.show()

In [ ]:
import numpy as np
import pandas as pd
plt.rcParams['font.family'] = 'Meiryo'

result_all = pd.DataFrame()

feature_names = [
    'velo_mean', # controlの方が大きい？
    'velo_max', # controlの方が大きい？
    'acce_mean', # controlの方が大きい？
    'acce_max', # controlの方が大きい？
    'saccade_count', # mciの方が大きい？
    'fixation_time_sum', # controlの方が大きい？
    'move_time_sum',
    'move_dis_sum',
    'saccade_move_dis_sum'
    ]
params = [
    [25000, 27000, 1864/2880, 385/1620, '4_1'], # コンテンツ4_1
    [28000, 30000, 1020/2880, 1234/1620, '4_2'], # コンテンツ4_2
    [40000, 46000, 0/2880, 0/1620, '6_1'], # コンテンツ6_1
    [47000, 53000, 0/2880, 0/1620, '6_2'], # コンテンツ6_2
    [54000, 60000, 0/2880, 0/1620, '6_3'], # コンテンツ6_3
    [64500, 66500, 0/2880, 0/1620, '7_1'], # コンテンツ7_1_b
    [70500, 72500, 0/2880, 0/1620, '7_2'], # コンテンツ7_2_b
    [83000, 85500, 0/2880, 0/1620, '7_3'], # コンテンツ7_3_b
    [96000, 98500, 0/2880, 0/1620, '7_4'], # コンテンツ7_4_b
    [109000, 111500, 0/2880, 0/1620, '7_5'], # コンテンツ7_5_b
    [91000, 96000, 0/2880, 0/1620, '8'], # コンテンツ8
    [104000, 109000, 0/2880, 0/1620, '9'], # コンテンツ9
    [115000, 119000, 0/2880, 0/1620, '10_1'], # コンテンツ10_1
    [120000, 124000, 0/2880, 0/1620, '10_2'], # コンテンツ10_2
    [125000, 129000, 0/2880, 0/1620, '10_3'], # コンテンツ10_3
    [130000, 134000, 0/2880, 0/1620, '10_4'], # コンテンツ10_4
    [135000, 139000, 0/2880, 0/1620, '10_5'], # コンテンツ10_5
    [142500, 147500, 0/2880, 0/1620, '11_1'], # コンテンツ11_1
    [147500, 152500, 0/2880, 0/1620, '11_2'], # コンテンツ11_2
    [214000, 219000, 0/2880, 0/1620, '14'], # コンテンツ14
    ]

for id in ids:
    df = dfs[dfs["id"] == id]
    df.rename(columns={'timestamp': 'TimeStamp'}, inplace=True)
    df['CloseEye'] = 0
    df['TimeStamp'] = df['TimeStamp'].astype(int)

    group_name = df['group_name'].unique()[0]

    frame_df, time_df = velocity_based_algorithm_summary(df, 'movie_pos_x', 'movie_pos_y', threshold=0.0003) # 元はthreshold=0.01475だが止まった判定が厳しいので閾値をあげる
    frame_df.rename(columns={'elapsed_times': 'timestamp', 'gaze_x':'movie_pos_x',  'gaze_y':'movie_pos_y', }, inplace=True)

    features_all = [group_name]
    feature_names_all = ['group_name']
    for p_index in range(len(params)):
        df_selected = frame_df[(frame_df['timestamp'] >= params[p_index][0]) & (frame_df['timestamp'] < params[p_index][1])]
        df_selected['movie_pos_x_interpolated'] = df_selected['movie_pos_x'].interpolate()
        df_selected['movie_pos_y_interpolated'] = df_selected['movie_pos_y'].interpolate()
        # 平均速度、最大速度[deg/ms]
        velo_mean, velo_max = get_velocity_info(df_selected)
        # 平均加速度、最大速度[deg/ms]
        acce_mean, acce_max = get_acceleration_info(df_selected)
        # サッケードの頻度[回/ms]
        saccade_count = get_saccade_count(df_selected)
        # # 方向転換の頻度[回/ms]
        # turn_count = get_turn_count(df_selected)
        # 停止時間の合計[ms]
        fixation_time_sum = get_fixtaion_time_sum(df_selected)
        # 速度V以上の時間の合計[ms]
        move_time_sum = get_move_time_sum(df_selected)
        # 移動距離の合計[deg]
        move_dis_sum = get_move_dis_sum(df_selected)
        # サッケード中の移動距離の合計[deg]
        saccade_move_dis_sum = get_saccade_move_dis_sum(df_selected)
        # # FFT(高周波成分が強いほど、視線の動きが速く変化していることを示す)
        # velo_fft = get_velo_fft(df_selected.dropna())
        features = [velo_mean, velo_max, acce_mean, acce_max, saccade_count, fixation_time_sum, move_time_sum, move_dis_sum, saccade_move_dis_sum]
        features_all.extend(features)
        feature_names_all.extend([f'{feature}_{params[p_index][4]}' for feature in feature_names])

        # グラフを描画
        # show_graph(df_selected, params, p_index)

    result = pd.DataFrame(data=[features_all], columns=feature_names_all, index=[id])
    result_all = pd.concat([result_all, result])
print(result_all)

NameError: name 'ids' is not defined

In [ ]:
from datetime import datetime
# result_all.to_csv(f"G:/共有ドライブ/MCI/v2/script/v2.1/検証中/新特徴量作成/output/視線の動きの俊敏さ_{datetime.now().strftime('%Y%m%d%H%M')}.csv")
result_all.to_csv(f"G:/共有ドライブ/MCI/v2/output/v2.1/kyoto/{period}/新特徴量/稲山さん/視線の動きの俊敏さ.csv")

# ここからは特徴量の評価

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import median_test
from scipy.stats import ranksums
from scipy.stats import brunnermunzel
from scipy.stats import ks_2samp

def interpret_p_value(p):
    if p < 0.05:
        print("有意差があります")
    else:
        print("有意差がありません")

def print_value(stat, p):
    print(f'統計量: {round(stat, 3)}')
    print(f'p値: {round(p, 5)}')

import seaborn as sns
from matplotlib.ticker import MaxNLocator
def show_hist_graph(column, df):
    _, ax = plt.subplots(figsize=(4,3))
    sns.histplot(data=df, ax=ax, x=column, hue='group_name', kde=True, palette=['blue', 'red'])
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.show()

def show_box_graph(column, df):
    custom_palette = {'CONTROL': 'blue', 'MCI': 'red'}
    _, ax = plt.subplots(figsize=(4, 3))
    sns.boxplot(x='group_name', y=column, data=df, ax=ax, palette=custom_palette, showmeans=True)
    plt.show()


evaluations_df = pd.DataFrame()
for i in range(len(list(result_all.columns))):
    labels = []
    values = []

    column = list(result_all.columns)[i]
    if column == 'group_name':
        continue
    labels.append('column')
    values.append(column)

    control_df = result_all[result_all['group_name'] == 'CONTROL'][column]
    mci_df = result_all[result_all['group_name'] == 'MCI'][column]
    control_df = control_df.dropna()
    mci_df = mci_df.dropna()

    # スチューデントのt検定を実行
    stat, p = ttest_ind(control_df, mci_df, equal_var=True)
    labels.append('stat_t検定')
    labels.append('p_t検定')
    values.append(stat)
    values.append(p)
    # if p < 0.001: # 有意差があるときだけグラフを描画
    #     show_hist_graph(column, result_all)
    #     show_box_graph(column, result_all)
    show_hist_graph(column, result_all)
    show_box_graph(column, result_all)

    # ウェルチのt検定を実行
    stat, p = ttest_ind(control_df, mci_df, equal_var=False)
    labels.append('stat_Welchst検定')
    labels.append('p_Welchst検定')
    values.append(stat)
    values.append(p)

    # マンホイットニーのU検定を実行
    setattr, p = mannwhitneyu(control_df, mci_df, alternative='two-sided')
    labels.append('stat_U検定')
    labels.append('p_U検定')
    values.append(stat)
    values.append(p)


    # 中央値検定
    stat, p, med, tbl = median_test(control_df, mci_df)
    labels.append('stat_中央値検定')
    labels.append('p_中央値検定')
    values.append(stat)
    values.append(p)


    # ウィルコクソンの順位和検定
    stat, p = ranksums(control_df, mci_df)
    labels.append('stat_ウィルコクソンの順位和検定')
    labels.append('p_ウィルコクソンの順位和検定')
    values.append(stat)
    values.append(p)


    # ブルンナー=ムンツェル検定
    stat, p = brunnermunzel(control_df, mci_df)
    labels.append('stat_ブルンナー=ムンツェル検定')
    labels.append('p_ブルンナー=ムンツェル検定')
    values.append(stat)
    values.append(p)


    # 2標本コルモゴロフ=スミルノフ検定
    stat, p = ks_2samp(control_df, mci_df)
    labels.append('stat_2標本コルモゴロフ=スミルノフ検定')
    labels.append('p_2標本コルモゴロフ=スミルノフ検定')
    values.append(stat)
    values.append(p)
    evaluation_df = pd.DataFrame(data=[values], columns=labels, index=[i])
    evaluations_df = pd.concat([evaluations_df, evaluation_df])

In [ ]:
evaluations_df.shape

(0, 0)

In [ ]:
evaluations_df.sort_values('p_t検定').head(20)[['column','p_t検定', 'p_Welchst検定','p_U検定']]

KeyError: 'p_t検定'

In [ ]:
def highlight_red(val):
    color = 'red' if val < 0.05 else ''
    return f'background-color: {color}'

# スタイルを適用して表示
styled_df = evaluations_df.style.applymap(highlight_red, subset=['p_t検定'])
styled_df

KeyError: "None of [Index(['p_t検定'], dtype='object')] are in the [columns]"

In [ ]:
correlation = result_all[result_all['group']=='control']['6-1_move_time_sum'].corr(result_all[result_all['group']=='control']['10-5_move_time_sum'])
print("control相関:", correlation)
correlation = result_all[result_all['group']=='mci']['6-1_move_time_sum'].corr(result_all[result_all['group']=='mci']['10-5_move_time_sum'])
print("mci相関:", correlation)
plt.scatter(result_all[result_all['group']=='control']['6-1_move_time_sum'],result_all[result_all['group']=='control']['10-5_move_time_sum'], c='blue')
plt.scatter(result_all[result_all['group']=='mci']['6-1_move_time_sum'],result_all[result_all['group']=='mci']['10-5_move_time_sum'], c='red')
plt.show()

correlation = result_all['6-1_move_time_sum'].corr(result_all['6-2_move_dis_sum'])
print("相関:", correlation)
plt.scatter(result_all[result_all['group']=='control']['6-1_move_time_sum'],result_all[result_all['group']=='control']['6-2_move_dis_sum'], c='blue')
plt.scatter(result_all[result_all['group']=='mci']['6-1_move_time_sum'],result_all[result_all['group']=='mci']['6-2_move_dis_sum'], c='red')
plt.show()

KeyError: 'group'

In [ ]:
from datetime import datetime
filtered_rows = evaluations_df[evaluations_df['p_t検定'] < 0.05]
selected_row_index = filtered_rows.index.tolist()
selected_row_index_and_group = selected_row_index.copy()
selected_row_index_and_group.insert(0, 0)
# result_all.iloc[:, selected_row_index_and_group].to_csv(f"G:/共有ドライブ/MCI/v2/script/v2.1/検証中/新特徴量作成/output/視線の動きの俊敏さ_{datetime.now().strftime('%Y%m%d%H%M')}.csv")

KeyError: 'p_t検定'

In [ ]:
import seaborn as sns
# グラフ描画領域の作成
plt.figure(figsize=(10, 10))
# plt.title('velo_mean, velo_max, acce_mean, acce_max, saccade_count, \nfixation_time_sum, move_time_sum, move_dis_sum, saccade_move_dis_sum')

# 相関係数のヒートマップを描画
sns.heatmap(
    result_all.iloc[:, selected_row_index].corr(), # p値<0.05列対象
    # result_all.iloc[:,1:].corr(), # 全列対象
    cmap="coolwarm",
    square=True,
    vmax=1,
    vmin=-1,)

plt.show()

NameError: name 'selected_row_index' is not defined

<Figure size 1000x1000 with 0 Axes>

### コメント
- ヒストグラムで見てよさそうだったのは4-1のfixation_time_sum
- saccade countはどのコンテンツ同士で相関が高い(視線の動き方は個人の傾向がある)
- move_dis_sumとsaccade_move_dis_sumはほぼイコールの特徴量になってしまったようで高相関→どちらかは無くてもいい